# Step 1: Install dependencies (required restart session)

In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py310_25.1.1-2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py310" --user

--2025-10-14 05:17:50--  https://repo.anaconda.com/miniconda/Miniconda3-py310_25.1.1-2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144058181 (137M) [application/octet-stream]
Saving to: ‘mini.sh’

mini.sh             100%[===================>] 137.38M  81.8MB/s    in 1.7s    

2025-10-14 05:17:52 (81.8 MB/s) - ‘mini.sh’ saved [144058181/144058181]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packag

In [ ]:
!git clone https://github.com/wjnwjn59/bizgen.git

Cloning into 'bizgen'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 79 (delta 16), reused 15 (delta 11), pack-reused 51 (from 1)
Receiving objects: 100% (79/79), 3.86 MiB | 19.77 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [ ]:
%cd bizgen

/content/bizgen


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: Wheel filename 'azure_ai_vision-0.8.0_alpha.1a0.dev0-py3-none-manylinux1_x86_64.whl' is not correctly normalised. Future versions of pip will raise the following error:
  Invalid wheel filename (invalid version): 'azure_ai_vision-0.8.0_alpha.1a0.dev0-py3-none-manylinux1_x86_64'
  
   pip 25.1 will enforce this behaviour change. A possible replacement is to rename the wheel to use a correctly normalised name (this may require updating the version in the project metadata). Discussion can be found at https://github.com/pypa/pip/issues/12938
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking long

In [ ]:
!pip install "numpy<2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 39.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


# Step 2: Download checkpoints

In [ ]:
%cd bizgen

/content/bizgen


In [ ]:
!git clone https://huggingface.co/PYY2001/BizGen

In [ ]:
!mv BizGen checkpoints

# Step 3: Prepare an infographic content from SQuADv2 data

In [ ]:
!pip install blingfire -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 18.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad_v2")

train_data = dataset["train"]

context = train_data[0]["context"]

qas = []
for example in train_data:
    if example["context"] == context:
        qas.append({
            "question": example["question"],
            "answers": example["answers"],
        })

print(f"Context:\n{context}\n")
print(f"Number of QAs for this context: {len(qas)}\n")

for i, qa in enumerate(qas):
    print(f"{i+1}. Q: {qa['question']}")
    print(f"   A: {qa['answers']['text']}")

Context:
Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

Number of QAs for this context: 15

1. Q: When did Beyonce start becoming popular?
   A: ['in the late 1990s']
2. Q: What areas did Beyonce compete in when she was growing up?
   A: ['singing and dancing']
3. Q: When did Beyonce leave Destiny's Child and become a solo singer?
   A: ['2003']
4. 

## Step 3.1: Get prompt 01

In [ ]:
import json
from jinja2 import Template
from blingfire import text_to_sentences

sents_raw = text_to_sentences(context).strip().split("\n")
sentences = [{"id": i + 1, "text": s} for i, s in enumerate(sents_raw)]

with open("content_des_stage_1.jinja", "r", encoding="utf-8") as f:
    template_text = f.read()

template = Template(template_text, trim_blocks=True, lstrip_blocks=True)
rendered_output = template.render(sents=sentences, qas=qas)

print(rendered_output)

Example output from prompt 01:
```
{
"summaries": [
{"id": 1, "summary": "Beyoncé Giselle Knowles-Carter, born September 4, 1981, is an American singer, songwriter, producer, and actress."},
{"id": 2, "summary": "Raised in Houston, Texas, she joined singing and dancing contests as a child and gained fame in the late 1990s as the lead singer of Destiny's Child."},
{"id": 3, "summary": "Managed by her father Mathew Knowles, Destiny's Child became one of the best-selling girl groups in history."},
{"id": 4, "summary": "During the group's hiatus, Beyoncé released her debut solo album 'Dangerously in Love' (2003), winning five Grammys and producing the number-one hits 'Crazy in Love' and 'Baby Boy'."}
]
}
```

In [2]:
# Put your output from prompt 1 here
stage_1_output = """
{
"summaries": [
{"id": 1, "summary": "Beyoncé Giselle Knowles-Carter, born September 4, 1981, is an American singer, songwriter, producer, and actress."},
{"id": 2, "summary": "Raised in Houston, Texas, she joined singing and dancing contests as a child and gained fame in the late 1990s as the lead singer of Destiny's Child."},
{"id": 3, "summary": "Managed by her father Mathew Knowles, Destiny's Child became one of the best-selling girl groups in history."},
{"id": 4, "summary": "During the group's hiatus, Beyoncé released her debut solo album 'Dangerously in Love' (2003), winning five Grammys and producing the number-one hits 'Crazy in Love' and 'Baby Boy'."}
]
}
"""

## Step 3.2: Get prompt 02 (required output from prompt 01)

In [ ]:
data = json.loads(stage_1_output)
items = data["summaries"]

with open("content_des_stage_2.jinja", "r", encoding="utf-8") as f:
    tmpl_text = f.read()

tmpl = Template(tmpl_text, trim_blocks=True, lstrip_blocks=True)
rendered_prompt_2 = tmpl.render(items=items)

print(rendered_prompt_2)

You are a dispassionate, rigorous, highly experienced infographic design planner. 
Our objective is to produce a single, coherent caption for a finished infographic that combines clear display text with complementary visual figures for layout-guided image generation. The workflow has three stages: Stage 1 creates concise per-sentence text items from the context; Stage 2 proposes visual figure ideas for each item; Stage 3 weaves all texts and figures into one cohesive, single-paragraph caption.

**Global constraints:** All proposed visuals must be rendered in a **cartoon/illustrative style**, and **no text/lettering/numerals/logos** may appear inside any visual.

You are in Stage 2 now; your task is to propose a **MAIN** visual idea for each text item from Stage 1 that conveys its meaning **without any lettering**. You **may also** provide **one SUPPORTIVE small figure** (optional), such as a simple abstract shape arrangement (e.g., “The image features a simple, abstract design with a b

Example output from prompt 02:
```
{
"figures": [
{"id": 1, "ideas": [
"A cartoon portrait of a confident woman with long flowing hair holds a microphone in one hand and stands on a softly lit stage, surrounded by musical notes and studio lights. She wears stylish attire and exudes a calm yet charismatic presence.",
"An abstract motif of a golden star overlapping a music note shape."
]},
{"id": 2, "ideas": [
"A young girl in casual clothes joyfully performs on a small stage, singing into a microphone while a few people applaud. Behind her, a bright sun and suburban houses evoke a warm Texas neighborhood.",
"A simple abstract swirl combining a sunburst and a small heart shape."
]},
{"id": 3, "ideas": [
"Three women in coordinated outfits perform synchronized dance moves under stage lights, while a man watches proudly from the side, suggesting guidance and management. The audience appears energetic and supportive in a concert-like setting.",
"A geometric arrangement of three interlocking circles symbolizing unity and teamwork."
]},
{"id": 4, "ideas": [
"The same woman now stands alone on a grand stage, surrounded by shining awards and musical symbols, conveying success and independence. She appears joyful, with spotlight beams emphasizing her solo stardom.",
"A stylized ribbon-like curve ascending upward, symbolizing rising achievement."
]}
]
}
```

In [3]:
# Put your output from prompt 2 here
stage_2_output = """
{
"figures": [
{"id": 1, "ideas": [
"A cartoon portrait of a confident woman with long flowing hair holds a microphone in one hand and stands on a softly lit stage, surrounded by musical notes and studio lights. She wears stylish attire and exudes a calm yet charismatic presence.",
"An abstract motif of a golden star overlapping a music note shape."
]},
{"id": 2, "ideas": [
"A young girl in casual clothes joyfully performs on a small stage, singing into a microphone while a few people applaud. Behind her, a bright sun and suburban houses evoke a warm Texas neighborhood.",
"A simple abstract swirl combining a sunburst and a small heart shape."
]},
{"id": 3, "ideas": [
"Three women in coordinated outfits perform synchronized dance moves under stage lights, while a man watches proudly from the side, suggesting guidance and management. The audience appears energetic and supportive in a concert-like setting.",
"A geometric arrangement of three interlocking circles symbolizing unity and teamwork."
]},
{"id": 4, "ideas": [
"The same woman now stands alone on a grand stage, surrounded by shining awards and musical symbols, conveying success and independence. She appears joyful, with spotlight beams emphasizing her solo stardom.",
"A stylized ribbon-like curve ascending upward, symbolizing rising achievement."
]}
]
}
"""

## Step 3.3: Get prompt 03 (required output from prompt 01 and prompt 02)

In [ ]:
s1 = json.loads(stage_1_output)
s2 = json.loads(stage_2_output)

merged = {it["id"]: {"id": it["id"], "summary": it["summary"], "ideas": []} for it in s1["summaries"]}
for it in s2["figures"]:
    if it["id"] in merged:
        merged[it["id"]]["ideas"] = it["ideas"]
merged_items = [merged[k] for k in sorted(merged.keys())]

with open("content_des_stage_3.jinja", "r", encoding="utf-8") as f:
    tmpl_text = f.read()

tmpl = Template(tmpl_text, trim_blocks=True, lstrip_blocks=True)
rendered_prompt_3 = tmpl.render(items=merged_items)

print(rendered_prompt_3)

Example output from prompt 3:
```
The image is an infographic titled "Beyoncé’s Rise to Stardom". The background features a smooth gradient of deep blue fading into warm gold, framed by subtle light rays and bordered with a thin decorative edge. The layout flows vertically from her early life at the top to her solo career at the bottom. At the top, "Beyoncé Giselle Knowles-Carter, born September 4, 1981, is an American singer, songwriter, producer, and actress." appears beside a cartoon portrait of a confident woman with long flowing hair holding a microphone on a softly lit stage, surrounded by musical notes and studio lights, with an abstract golden star overlapping a music note shape nearby. Below this, "Raised in Houston, Texas, she joined singing and dancing contests as a child and gained fame in the late 1990s as the lead singer of Destiny's Child." is illustrated by a cheerful young girl performing on a small neighborhood stage under a bright sun with warm houses in the background, accompanied by a simple swirl of a sunburst and heart motif. In the center section, "Managed by her father Mathew Knowles, Destiny's Child became one of the best-selling girl groups in history." is paired with three women performing synchronized dance moves under stage lights while a proud man watches, the energy mirrored by a geometric pattern of three interlocking circles suggesting unity and teamwork. At the bottom, "During the group's hiatus, Beyoncé released her debut solo album 'Dangerously in Love' (2003), winning five Grammys and producing the number-one hits 'Crazy in Love' and 'Baby Boy'." is visualized by the same woman standing alone on a grand stage surrounded by shining awards and musical symbols under bright spotlights, enhanced by a stylized ribbon-like curve rising upward to symbolize her achievements. The overall style of the image is colorful, balanced, and elegantly illustrative, blending biographical storytelling with symbolic visual harmony.
```

In [4]:
# Put your output from prompt 3 here
stage_3_output = """
The image is an infographic titled "Beyoncé’s Rise to Stardom". The background features a smooth gradient of deep blue fading into warm gold, framed by subtle light rays and bordered with a thin decorative edge. The layout flows vertically from her early life at the top to her solo career at the bottom. At the top, "Beyoncé Giselle Knowles-Carter, born September 4, 1981, is an American singer, songwriter, producer, and actress." appears beside a cartoon portrait of a confident woman with long flowing hair holding a microphone on a softly lit stage, surrounded by musical notes and studio lights, with an abstract golden star overlapping a music note shape nearby. Below this, "Raised in Houston, Texas, she joined singing and dancing contests as a child and gained fame in the late 1990s as the lead singer of Destiny's Child." is illustrated by a cheerful young girl performing on a small neighborhood stage under a bright sun with warm houses in the background, accompanied by a simple swirl of a sunburst and heart motif. In the center section, "Managed by her father Mathew Knowles, Destiny's Child became one of the best-selling girl groups in history." is paired with three women performing synchronized dance moves under stage lights while a proud man watches, the energy mirrored by a geometric pattern of three interlocking circles suggesting unity and teamwork. At the bottom, "During the group's hiatus, Beyoncé released her debut solo album 'Dangerously in Love' (2003), winning five Grammys and producing the number-one hits 'Crazy in Love' and 'Baby Boy'." is visualized by the same woman standing alone on a grand stage surrounded by shining awards and musical symbols under bright spotlights, enhanced by a stylized ribbon-like curve rising upward to symbolize her achievements. The overall style of the image is colorful, balanced, and elegantly illustrative, blending biographical storytelling with symbolic visual harmony.
"""

# Step 4: Prepare JSON for BizGen

In [5]:
s1 = json.loads(stage_1_output)
s2 = json.loads(stage_2_output)
full_caption = stage_3_output.strip()

merged = {it["id"]: {"id": it["id"], "summary": it["summary"], "ideas": []} for it in s1["summaries"]}
for it in s2["figures"]:
    if it["id"] in merged:
        merged[it["id"]]["ideas"] = it["ideas"]

merged_figures = [merged[k] for k in sorted(merged.keys())]

bizgen_input = [{
    "full_image_caption": full_caption,
    "background_caption": "Modern gradient background transitioning from light blue to white",
    "figures": merged_figures
}]

In [6]:
# Import and use the data generator with JSON string input
import generate_bizgen_data

# Convert bizgen_input to JSON string (the function expects JSON string, not dict)
import json
json_string_input = json.dumps(bizgen_input)

# Generate BizGen format data from the JSON string
bizgen_data = generate_bizgen_data.merge_narrator_data_from_string(
    input_json_string=json_string_input,
    max_samples=1,
    output_path="meta/test_infographics.json"
)

# Verify the generated file
!ls -la meta/test_infographics.json

=== Merge Stage Narrator ===
Processing narrator-generated infographic data from JSON string...
Loading required data files...
  - Loaded 100 bbox sets
  - Loaded 138 colors
  - Loaded 988 fonts
  - Parsed 1 infographic entries from JSON string
  - Input format validation: Format is valid

Processing infographic data...
Auto-scaling small text bboxes for wiki 1...
Validating canvas bounds for wiki 1...
Layout quality warning for wiki 1: score=0.60

Success! Generated 1 wiki entries
Output saved to: meta/test_infographics.json
Data format matches merge_stage_narrator.py output
-rw-rw-r-- 1 thinhnp thinhnp 10304 Thg 10 25 14:06 meta/test_infographics.json


# Step 5: Inference

In [7]:
%%time
!python inference.py \
--ckpt_dir checkpoints/lora/infographic \
--output_dir infographic \
--sample_list meta/test_infographics.json

Traceback (most recent call last):
  File "/home/thinhnp/hf_vqa/src/data/bizgen/inference.py", line 9, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'
CPU times: user 6.25 ms, sys: 9.11 ms, total: 15.4 ms
Wall time: 793 ms
